In [0]:


configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "#",
"fs.azure.account.oauth2.client.secret": '#',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/031204db-144a-4858-897f-7c9343d308ec/oauth2/token"}


dbutils.fs.mount(
source = "abfss://olympic-data@tokyolymic.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyolymic",
extra_configs = configs)
  

True

In [0]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyolymic/raw/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyolymic/raw/coaches.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyolymic/raw/teams.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyolymic/raw/medals.csv")
entities_gender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyolymic/raw/entities_gender.csv")

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import sum
from pyspark.sql.functions import avg



In [0]:
entities_gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
entities_gender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
entities_gender = entities_gender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
        .withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
medals.printSchema() #changing schema from reading the csv file with option(infer_schema,true)

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
#top countries winning gold medal
top_counties_by_gold = medals.orderBy("Gold", ascending = False).select("TeamCountry", "Gold")
top_counties_by_gold.show()

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
# filtering based on country \
athletes = athletes.filter(col("Country") == "Spain").show()

+--------------------+-------+-------------------+
|          PersonName|Country|         Discipline|
+--------------------+-------+-------------------+
|         ABAD Nestor|  Spain|Artistic Gymnastics|
|      ABALDE Alberto|  Spain|         Basketball|
|       ABALDE Tamara|  Spain|         Basketball|
|         ABIAN Pablo|  Spain|          Badminton|
|ABRINES REDONDO A...|  Spain|         Basketball|
|AGUINAGALDE AQUIZ...|  Spain|           Handball|
|        AGUIRRE Unai|  Spain|         Water Polo|
|     ALARZA Fernando|  Spain|          Triathlon|
| ALEGRE BIOSCA David|  Spain|             Hockey|
|    ALONSO Alejandro|  Spain|             Hockey|
|ALVAREZ AZNAR Edu...|  Spain|         Equestrian|
|       ANDUJAR Pablo|  Spain|             Tennis|
|         ARCE Daniel|  Spain|          Athletics|
|  ARCOS POVEDA Paula|  Spain|           Handball|
|        ARENAS Julio|  Spain|          Athletics|
|AREVALO ALCON Alb...|  Spain|             Diving|
|      AREVALO Carlos|  Spain| 

In [0]:
#export into csv\
medals.write.format("csv").option("header","true").save("/mnt/tokyolymic/processed/medals.csv")

In [0]:
total_medals = medals.select(
    sum("Gold").alias("Total_Gold"),
    sum("Silver").alias("Total_Silver"),
    sum("Bronze").alias("Total_Bronze"),
    sum("Total").alias("Total_Medals")
)
total_medals.show()

+----------+------------+------------+------------+
|Total_Gold|Total_Silver|Total_Bronze|Total_Medals|
+----------+------------+------------+------------+
|       340|         338|         402|        1080|
+----------+------------+------------+------------+

